In [0]:
%run ../utils

In [0]:
dbutils.widgets.text("adhoc_date", "")

In [0]:
adhoc_date = dbutils.widgets.get("adhoc_date")
# display(adhoc_date)

In [0]:
date = genrate_yesterdays_date(adhoc_date)
print(f"current job is running for the date: {date}")

# create dataframe on input dataset of customer profile
df_transaction = spark.read.csv(f"/mnt/bronze-layer/{date}/customer_transaction.csv", header=True)
print(df_transaction.show())
print(df_transaction.printSchema())

In [0]:

# Defined window specification to get unique record based on customer_id, transaction_id and transaction_date
window_spec = Window.partitionBy("customer_id", "transaction_id", "transaction_date") \
        .orderBy("transaction_id")

# apply row_number function above window specification
df_transaction = df_transaction.withColumn("row_no", row_number().over(window_spec))
# print(df_transaction.show())

# bad_record
duplicate_bad_df = df_transaction.filter("row_no != 1").drop("row_no")

# select unique record based on row_number and drop column
df_transaction = df_transaction.filter("row_no = 1").drop("row_no")
print(df_transaction.show())




In [0]:

regex_expression = "^[a-zA-Z0-9 ]+$"

#bad_record
special_char_bad_df = df_transaction.filter(~col("category").rlike(regex_expression))


df_transaction = df_transaction.filter(col("category").rlike(regex_expression))

df_transaction.show()




In [0]:



# is_large_transaction for transactions > $1000.
df_transaction = df_transaction.withColumn("is_large_transaction",when(col("amount").cast(DecimalType(18, 2)) > 1000, True).otherwise(False))
# df_transaction.show()

# year, month, day for easier aggregation.
df_transaction = df_transaction.withColumn("transaction_year", year(col("transaction_date").cast(TimestampType()))) \
        .withColumn("transaction_month", month(col("transaction_date").cast(TimestampType()))) \
        .withColumn("transaction_day", dayofmonth(col("transaction_date").cast(TimestampType())))

df_transaction.show()

    

In [0]:
# write data to silver layer
df_transaction.coalesce(1).write.mode("overwrite").parquet(f"/mnt/silver-layer/{date}/customer_transaction")

#witre bad record with uni of all bad record
duplicate_bad_df.union(special_char_bad_df).write.mode("overwrite").csv(fr"mnt/silver-layer/bad_record/{date}/customer_transaction", header = True)